In [ ]:
import cv2
import SimpleITK as sitk
import tifffile as tf
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import time

FALTA PULIR EL BINARIZE Y EL OPEN PARA QUE SEA MAS PRECISO

In [ ]:
def encontrar_contornos(canal,stack_resized):

    contornos, _ = cv2.findContours(canal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    regiones_interes = []
    for contorno in contornos:
        print("Contorno detectado")
        x, y, w, h = cv2.boundingRect(contorno)
        print(x,y,w,h)

        region = stack_resized[y:y+h, x:x+w]
        regiones_interes.append(region)
    return regiones_interes,contornos

In [ ]:
def sliding_window(imagen_bin,imagen, window_size):
    valid_patches = []
    for y in range(0, imagen_bin.shape[0] - window_size[0], 7 ):
        for x in range(0, imagen_bin.shape[1] - window_size[1], 7 ):
  
            patch = imagen_bin[y:y+window_size[0], x:x+window_size[1]]

            white_pixels = cv2.countNonZero(patch)
            black_pixels = patch.size - white_pixels

            if white_pixels > 0 and black_pixels > 0:
                patch_imagen_original = imagen[y:y+window_size[0], x:x+window_size[1]]
                valid_patches.append(patch_imagen_original)
    return valid_patches


In [ ]:
def extract_patches(rute):
    tif_stack = tf.imread(rute)
    print("The shape of your image is:",tif_stack.shape)
    print("Resizing image...")
    alto,ancho,canales= tif_stack.shape
    antiguo_aspect_ratio= ancho/alto
    
    nuevo_ancho = int(ancho/100)  
    nuevo_alto = int(alto/100)  

    nuevo_aspect_ratio= nuevo_ancho/ nuevo_alto

    # Redimensiona la imagen manteniendo la relación de aspecto
    stack_resized = cv2.resize(tif_stack, (nuevo_ancho, nuevo_alto))
    
    print("The new shape is:",stack_resized.shape)
    time.sleep(1)
    print("Practically we preserve all the aspect ratio when resizing since the original was:",antiguo_aspect_ratio,"and the new one is:", nuevo_aspect_ratio)
    time.sleep(1)
    print("Plotting image...")
    time.sleep(1)
    plt.imshow(stack_resized)
    plt.show()
    
    print("Converting image from BGR to HSV...")
    imagen_hsv = cv2.cvtColor(stack_resized, cv2.COLOR_BGR2HSV)
    time.sleep(1)
    print("Extracting Hue channel...")
    canal= imagen_hsv[:,:,:1]
    time.sleep(1)
    print("Canal extracted! Shape:",canal.shape)
    time.sleep(1)
    print("Visualizing..")
    plt.imshow(canal)
    plt.show()
    print("Preparing to extract contours..")
    regiones_interes,contornos=encontrar_contornos(canal,stack_resized)
  
    imagen_con_contornos = stack_resized.copy()

    print("Visualizing extracted contours..")
    # Dibuja los contornos en la imagen
    cv2.drawContours(imagen_con_contornos, contornos, -1, (255,0, 0), 9)
    plt.imshow(imagen_con_contornos)
    plt.axis('off')  
    plt.show()

    time.sleep(1)
    for i in range(len(regiones_interes)):
        crop= regiones_interes[i]
        time.sleep(1)
        print("Printing crop number:",i)
        plt.imshow(crop)
        plt.axis('off') 
        plt.show()
        time.sleep(1)

    print("Time to process the cropped images to extract patches..")

    for i in range(len(regiones_interes)):
        print("Extracting saturation channel from",i,"crop...")
        imagen_hsv = cv2.cvtColor(regiones_interes[i], cv2.COLOR_BGR2HSV)
        canal_crop= imagen_hsv[:,:,1]
        time.sleep(1)
        plt.imshow(canal_crop, cmap="gray")
        plt.axis('off') 
        plt.show()

        time.sleep(1)
        print("Binarizing image...")
        umbral, imagen_binaria = cv2.threshold(canal_crop, 35, 180, cv2.THRESH_BINARY)
        time.sleep(1)
        plt.imshow(imagen_binaria, cmap="gray")
        plt.axis('off') 
        plt.show()

        time.sleep(1)

        print("Applying morphological operation to avoid holes and noise... (opening)")
        # Define el kernel para el open
        kernel = np.ones((2, 2), np.uint8)  

        
        imagen_abierta = cv2.morphologyEx(imagen_binaria, cv2.MORPH_OPEN, kernel)
        time.sleep(1)

        plt.imshow(imagen_abierta, cmap="gray")
        plt.axis('off') 
        plt.show()
        time.sleep(1)

        print("Defining window for sliding window process..")
        window_size=(15,15)
        patches=sliding_window(imagen_abierta,regiones_interes[i],window_size)
        time.sleep(1)
        print("Got it! Patches obtained:",len(patches))
        time.sleep(1)

        print("Visualizing 5 first patches..")
        for i in range (0,5):
            plt.imshow(patches[i])
            plt.axis('off')  
            plt.show()


    time.sleep(1)     
    print("Patches extraction process finished successfully!")
    #return patches


        



In [ ]:
tif_file = "/fhome/mapsiv/QuironHelico/WSI_Example/B22-25-HP.tiff"
extract_patches(tif_file)

In [ ]:
tif_file = "/fhome/mapsiv/QuironHelico/WSI_Example/B22-35-HP.tiff"
extract_patches(tif_file)

In [ ]:
tif_file = "/fhome/mapsiv/QuironHelico/WSI_Example/B22-31-HP.tiff"
extract_patches(tif_file)

In [ ]:
tif_stack.shape

In [ ]:
alto,ancho,canales= tif_stack.shape
alto,ancho

In [ ]:
type(tif_stack)

In [ ]:
antiguo_aspect_ratio= ancho/alto

nuevo_ancho = int(ancho/100) 
nuevo_alto = int(alto/100)  

nuevo_aspect_ratio= nuevo_ancho/ nuevo_alto


stack_resized = cv2.resize(tif_stack, (nuevo_ancho, nuevo_alto))


print("Prácticamente conservamos todo el aspect ratio redimensionando ya que el antiguo era:",antiguo_aspect_ratio,"y el nuevo es:", nuevo_aspect_ratio)

In [ ]:
plt.imshow(stack_resized)
plt.show()

In [ ]:
plt.imshow(stack_resized)
plt.show()

In [ ]:

imagen_hsv = cv2.cvtColor(stack_resized, cv2.COLOR_BGR2HSV)

canal= imagen_hsv[:,:,:1]

canal.shape

In [ ]:
plt.imshow(canal)
plt.show()

In [ ]:

contornos, _ = cv2.findContours(canal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


regiones_interes = []
for contorno in contornos:
    print("Contorno detectado")
    x, y, w, h = cv2.boundingRect(contorno)
    print(x,y,w,h)

    region = stack_resized[y:y+h, x:x+w]
    regiones_interes.append(region)

In [ ]:

imagen_con_contornos = stack_resized.copy()

cv2.drawContours(imagen_con_contornos, contornos, -1, (255,0, 0), 9)
plt.imshow(imagen_con_contornos)
plt.axis('off')  
plt.show()

In [ ]:
len(regiones_interes)

In [ ]:
crop1= regiones_interes[0]
crop2= regiones_interes[1]
crop3= regiones_interes[2]

In [ ]:
plt.imshow(crop1)
plt.axis('off') 
plt.show()

In [ ]:
plt.imshow(crop2)
plt.axis('off') 
plt.show()

In [ ]:
plt.imshow(crop3)
plt.axis('off') 
plt.show()

In [ ]:

imagen_hsv = cv2.cvtColor(crop1, cv2.COLOR_BGR2HSV)

canal_crop= imagen_hsv[:,:,1]

canal_crop.shape

In [ ]:
plt.imshow(canal_crop, cmap="gray")
plt.axis('off') 
plt.show()

In [ ]:

umbral, imagen_binaria = cv2.threshold(canal_crop, 10, 180, cv2.THRESH_BINARY)



In [ ]:
plt.imshow(imagen_binaria, cmap="gray")
plt.axis('off') 
plt.show()

In [ ]:

kernel = np.ones((2, 2), np.uint8)  


imagen_abierta = cv2.morphologyEx(imagen_binaria, cv2.MORPH_OPEN, kernel)

plt.imshow(imagen_abierta, cmap="gray")
plt.axis('off') 
plt.show()

In [ ]:

contornos, _ = cv2.findContours(imagen_abierta, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

regiones_interes = []
for contorno in contornos:
    print("Contorno detectado")
    x, y, w, h = cv2.boundingRect(contorno)
    print(x,y,w,h)

    region = stack_resized[y:y+h, x:x+w]
    regiones_interes.append(region)

In [ ]:

imagen_con_contornos = crop1.copy()

cv2.drawContours(imagen_con_contornos, contornos, -1, (255,0, 0), 1)
plt.imshow(imagen_con_contornos)
plt.axis('off')  # Desactiva los ejes
plt.show()

In [ ]:
def sliding_window(imagen_bin,imagen, window_size):
    valid_patches = []
    for y in range(0, imagen_bin.shape[0] - window_size[0] + 10):
        for x in range(0, imagen_bin.shape[1] - window_size[1] + 10):
            # Extrae el patch actual
            patch = imagen_bin[y:y+window_size[0], x:x+window_size[1]]

            # Calcula la proporción de píxeles blancos en el patch
            white_pixels = np.sum(patch == 255)  # Suponiendo que el blanco es 255 en una imagen binarizada

            # Calcula la proporción de píxeles negros en el patch
            black_pixels = np.sum(patch == 0)  # Suponiendo que el negro es 0 en una imagen binarizada
            # Verifica si el patch contiene tanto blanco como negro
            if white_pixels > 0:
                patch_imagen_original = imagen[y:y+window_size[0], x:x+window_size[1]]
                valid_patches.append(patch_imagen_original)
            
    return valid_patches


In [ ]:
def sliding_window(imagen_bin,imagen, window_size):
    valid_patches = []
    for y in range(0, imagen_bin.shape[0] - window_size[0], 7 ):
        for x in range(0, imagen_bin.shape[1] - window_size[1], 7 ):
            # Extrae el patch actual
            patch = imagen_bin[y:y+window_size[0], x:x+window_size[1]]

            white_pixels = cv2.countNonZero(patch)
            black_pixels = patch.size - white_pixels

            if white_pixels > 0 and black_pixels > 0:
                patch_imagen_original = imagen[y:y+window_size[0], x:x+window_size[1]]
                valid_patches.append(patch_imagen_original)
    return valid_patches


In [ ]:
window_size=(15,15)
patches=sliding_window(imagen_abierta,crop1,window_size)



In [ ]:
len(patches)

In [ ]:
for i in range (0,50):
    plt.imshow(patches[i])
    plt.axis('off')  # Desactiva los ejes
    plt.show()